In [ ]:
# anilist_fetch_for_hit_prediction.py
# ------------------------------------------------------------
# 目标：
# 1) 抓取 2016-2025 年（近10年）的日本动画条目（训练集）
# 2) 抓取 2026 年 1 月（WINTER 2026）待上映/当季条目（预测候选集）
# 3) 输出两个 CSV：anilist_anime_2016_2025.csv、anilist_winter_2026_candidates.csv
# ------------------------------------------------------------

import time
import math
import json
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timezone

ANILIST_ENDPOINT = "https://graphql.anilist.co"

# 统一列名
def normalize_columns(df):
    df = df.copy()
    df.columns = (
        df.columns.str.strip()
                  .str.lower()
                  .str.replace(" ", "_")
                  .str.replace("(", "")
                  .str.replace(")", "")
                  .str.replace("-", "_")
    )
    return df

# 文本清洗
def clean_text(x):
    if pd.isna(x):
        return x
    x = str(x)
    return (
        x.replace("\n", " ")
         .replace("<i>", "")
         .replace("</i>", "")
         .replace("<br>", " ")
         .strip()
    )

# Genres list 清洗
def parse_genres(x):
    if pd.isna(x):
        return []
    return [g.strip() for g in str(x).split(",") if g.strip()]


# ---------------------------
# GraphQL 查询（媒体/作品）
# ---------------------------
MEDIA_QUERY = """
query ($page:Int, $perPage:Int, $season:MediaSeason, $seasonYear:Int) {
  Page(page:$page, perPage:$perPage) {
    pageInfo { total perPage currentPage lastPage hasNextPage }
    media(
      type: ANIME
      countryOfOrigin: "JP"
      isAdult: false
      season: $season
      seasonYear: $seasonYear
      sort: [POPULARITY_DESC]
    ) {
      id
      idMal
      title { romaji english native userPreferred }
      season
      seasonYear
      startDate { year month day }
      endDate   { year month day }
      episodes
      duration
      format
      status
      source
      countryOfOrigin
      averageScore
      meanScore
      popularity
      favourites
      trending
      genres
      tags { name rank category isGeneralSpoiler isMediaSpoiler }
      studios {
        edges { isMain }
        nodes { id name isAnimationStudio }
      }
      rankings {
        rank
        type
        year
        season
        allTime
        context
      }
      externalLinks { site url }
      trailer { id site }
      coverImage { large medium }
    }
  }
}
"""

# ---------------------------
# GraphQL 查询（放送日程）
# * 用于 2026-01 的候选，过滤 1 月内的排程（秒级时间戳）
# ---------------------------
AIRING_QUERY = """
query ($page:Int, $perPage:Int, $notYet:Boolean, $from:Int, $to:Int) {
  Page(page:$page, perPage:$perPage) {
    pageInfo { total perPage currentPage lastPage hasNextPage }
    airingSchedules(airingAt_greater:$from, airingAt_lesser:$to, notYetAired:$notYet) {
      id
      airingAt
      timeUntilAiring
      episode
      media {
        id
        title { romaji english native userPreferred }
        season
        seasonYear
        format
        status
        source
        averageScore
        popularity
        favourites
      }
    }
  }
}
"""

def run_query(query, variables, max_retries=4, sleep_s=0.7):
    """调用 AniList GraphQL，带重试和简单速率控制。"""
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.post(ANILIST_ENDPOINT, json={"query": query, "variables": variables}, timeout=30)
            if r.status_code == 200:
                data = r.json()
                if "errors" in data:
                    raise RuntimeError(data["errors"])
                return data["data"]
            elif r.status_code in (429, 502, 503):
                # 限流或网关问题，退避
                wait = sleep_s * attempt * 2
                time.sleep(wait)
            else:
                r.raise_for_status()
        except Exception as e:
            if attempt == max_retries:
                raise
            time.sleep(sleep_s * attempt)
    raise RuntimeError("Unreachable")

def fetch_season(year: int, season: str, per_page=50):
    """抓取某一年某个季（WINTER/SPRING/SUMMER/FALL）的所有日本动画。"""
    page = 1
    all_media = []
    while True:
        payload = {"page": page, "perPage": per_page, "season": season, "seasonYear": year}
        data = run_query(MEDIA_QUERY, payload)
        info = data["Page"]["pageInfo"]
        media = data["Page"]["media"]
        all_media.extend(media)
        if not info["hasNextPage"]:
            break
        page += 1
        time.sleep(0.4)  # 轻微降速，友好一点
    return all_media

def normalize_media_rows(media_list):
    """把 GraphQL 返回的 media 列表摊平成表格行。"""
    rows = []
    for m in media_list:
        studios = m.get("studios", {}) or {}
        nodes = studios.get("nodes", []) or []
        edges = studios.get("edges", []) or []
        main_studio = None
        if nodes and edges and len(nodes) == len(edges):
            for node, edge in zip(nodes, edges):
                if edge.get("isMain"):
                    main_studio = node.get("name")
                    break
        if main_studio is None and nodes:
            main_studio = nodes[0].get("name")

        row = {
            "id": m.get("id"),
            "idMal": m.get("idMal"),
            "title_romaji": (m.get("title") or {}).get("romaji"),
            "title_english": (m.get("title") or {}).get("english"),
            "title_native": (m.get("title") or {}).get("native"),
            "season": m.get("season"),
            "seasonYear": m.get("seasonYear"),
            "startDate": "-".join(str((m.get("startDate") or {}).get(k) or "") for k in ["year","month","day"]),
            "endDate": "-".join(str((m.get("endDate") or {}).get(k) or "") for k in ["year","month","day"]),
            "episodes": m.get("episodes"),
            "duration": m.get("duration"),
            "format": m.get("format"),
            "status": m.get("status"),
            "source": m.get("source"),
            "averageScore": m.get("averageScore"),
            "meanScore": m.get("meanScore"),
            "popularity": m.get("popularity"),
            "favourites": m.get("favourites"),
            "trending": m.get("trending"),
            "genres": "|".join(m.get("genres") or []),
            "tags": "|".join([t.get("name") for t in (m.get("tags") or [])]),
            "mainStudio": main_studio,
            "rankings_json": json.dumps(m.get("rankings") or [], ensure_ascii=False),
            "externalLinks_json": json.dumps(m.get("externalLinks") or [], ensure_ascii=False),
        }
        rows.append(row)
    return rows

def fetch_training_dataset(year_start=2016, year_end=2025):
    """抓取近10年（含）全部四季数据并保存 CSV。"""
    seasons = ["WINTER", "SPRING", "SUMMER", "FALL"]
    collected = []
    for y in range(year_start, year_end + 1):
        for s in seasons:
            print(f"Fetching {y} {s} ...")
            media = fetch_season(y, s)
            collected.extend(normalize_media_rows(media))
    df = pd.DataFrame(collected)
    df.drop_duplicates(subset=["id"], inplace=True)
    out = f"anilist_anime_{year_start}_{year_end}.csv"
    df.to_csv(out, index=False, encoding="utf-8-sig")
    print(f"✅ Saved training dataset: {out}  (rows={len(df)})")
    return df

# ---------------------------
# 2026-01 候选（当季 WINTER 2026 + 播放日程）
# ---------------------------
def unix_ts(dt_str: str):
    """'YYYY-MM-DD' → Unix 秒"""
    dt = datetime.strptime(dt_str, "%Y-%m-%d").replace(tzinfo=timezone.utc)
    return int(dt.timestamp())

def fetch_airing_window(start_date: str, end_date: str, not_yet=True, per_page=50):
    """按时间窗口抓取放送日程（用于 2026-01）。"""
    page = 1
    all_rows = []
    from_ts = unix_ts(start_date)
    to_ts = unix_ts(end_date)
    while True:
        payload = {
            "page": page,
            "perPage": per_page,
            "from": from_ts,
            "to": to_ts,
            "notYet": bool(not_yet)
        }
        data = run_query(AIRING_QUERY, payload)
        info = data["Page"]["pageInfo"]
        items = data["Page"]["airingSchedules"]
        for it in items:
            m = it.get("media") or {}
            row = {
                "airing_id": it.get("id"),
                "airingAt": it.get("airingAt"),
                "episode": it.get("episode"),
                "timeUntilAiring": it.get("timeUntilAiring"),
                "media_id": m.get("id"),
                "title_romaji": (m.get("title") or {}).get("romaji"),
                "title_english": (m.get("title") or {}).get("english"),
                "season": m.get("season"),
                "seasonYear": m.get("seasonYear"),
                "format": m.get("format"),
                "status": m.get("status"),
                "source": m.get("source"),
                "averageScore": m.get("averageScore"),
                "popularity": m.get("popularity"),
                "favourites": m.get("favourites"),
            }
            all_rows.append(row)
        if not info["hasNextPage"]:
            break
        page += 1
        time.sleep(0.4)
    df = pd.DataFrame(all_rows)
    return df

def fetch_winter_2026_candidates():
    """
    最终修正版：
    - 合并 season fetch + airing schedule
    - 自动选择英文标题作为主 title（title_english）
    - 保证字段统一且不会出现缺失
    """
    print("🔍 Fetching 2026 WINTER candidates (enhanced with English titles)...")

    # ----------------------
    # Step 1: Season fetch
    # ----------------------
    season_raw = fetch_season(2026, "WINTER") 
    df_season = pd.DataFrame(normalize_media_rows(season_raw))
    print(f"Season-based winter 2026: {len(df_season)} entries")

    # ----------------------
    # Step 2: Airing schedule (Jan 1 – Feb 1, 2026)
    # ----------------------
    df_air = fetch_airing_window("2026-01-01", "2026-02-01", not_yet=True)
    print(f"Airing schedule hits: {len(df_air)} entries")

    # ----------------------
    # Step 3: 标准化列名，确保 id 存在
    # ----------------------
    df_season = normalize_columns(df_season)
    df_air = normalize_columns(df_air)

    for df in [df_air, df_season]:
        if "id" not in df.columns:
            id_candidates = [c for c in df.columns if "id" in c.lower()]
            if id_candidates:
                df.rename(columns={id_candidates[0]: "id"}, inplace=True)

    if "id" not in df_air.columns or "id" not in df_season.columns:
        raise KeyError("❌ Missing id in merged datasets!")

    # ----------------------
    # Step 4: Merge datasets on id
    # ----------------------
    df_all = pd.merge(df_air, df_season, on="id", how="left", suffixes=("_air", "_media"))
    print(f"Merged dataset: {len(df_all)} rows")

    # ----------------------
    # Step 5: 选择统一标题：title_english
    # ----------------------
    def choose_english(row):
        """
        优先顺序：
        1) title_english_media（season fetch 更完整）
        2) title_english_air
        """
        if pd.notna(row.get("title_english_media")):
            return row["title_english_media"]
        if pd.notna(row.get("title_english_air")):
            return row["title_english_air"]
        return None

    df_all["title_english"] = df_all.apply(choose_english, axis=1)

    # ----------------------
    # Step 6: 过滤合规冬番（去掉旧番/非冬季/多季连载等）
    # ----------------------
    df_all["seasonyear_media"] = pd.to_numeric(df_all.get("seasonyear_media", np.nan), errors="coerce")
    df_all["episodes_media"] = pd.to_numeric(df_all.get("episodes_media", np.nan), errors="coerce")

    df_all = df_all[
        (df_all["seasonyear_media"] >= 2025)
        & (df_all["episodes_media"].fillna(0) <= 50)
    ].copy()

    # ----------------------
    # Step 7: 辅助字段
    # ----------------------
    df_all["has_airing_schedule"] = df_all["airingat"].notna().astype(int)
    df_all["estimated_start"] = pd.to_datetime(df_all["airingat"], unit="s", errors="coerce")

    # ----------------------
    # Step 8: 输出（只保留必要字段）
    # ----------------------
    keep_cols = [
        "id",
        "title_english",
        "season_media",
        "seasonyear_media",
        "format_media",
        "status_media",
        "source_media",
        "average_score_media",
        "popularity_media",
        "favourites_media",
        "genres_media",
        "mainstudio_media",
        "episodes_media",
        "duration_media",
        "has_airing_schedule",
        "estimated_start",
    ]

    df_export = df_all[keep_cols]
    df_export.rename(columns=lambda c: c.replace("_media", ""), inplace=True)

    out = "anilist_winter_2026_candidates_filtered.csv"
    df_export.to_csv(out, index=False, encoding="utf-8-sig")

    print(f"🎉 Final 2026 candidates saved: {out} (rows={len(df_export)})")
    return df_export




# ---------------------------
# 主流程
# ---------------------------
if __name__ == "__main__":
    fetch_training_dataset(2016, 2025)
    print("🎉 Done.")


In [3]:
import requests
import csv

API_URL = "https://graphql.anilist.co"

def run_query(query, variables=None):
    """执行 AniList GraphQL 查询"""
    headers = {"Content-Type": "application/json"}
    response = requests.post(API_URL, json={"query": query, "variables": variables or {}}, headers=headers)
    response.raise_for_status()
    data = response.json()
    if "errors" in data:
        raise Exception(data["errors"])
    return data["data"]

def get_winter_2026_anime(page=1, per_page=50):
    """获取 2026 年冬季（1月）新番"""
    query = """
    query ($page: Int, $perPage: Int) {
      Page(page: $page, perPage: $perPage) {
        pageInfo {
          hasNextPage
          currentPage
        }
        media(
          type: ANIME
          season: WINTER
          seasonYear: 2026
          sort: START_DATE
        ) {
          id
          title {
            romaji
            english
            native
            userPreferred
          }
          description(asHtml: false)
          format
          episodes
          duration
          startDate { year month day }
          studios(isMain: true) {
            nodes { name }
          }
          genres
          averageScore
          popularity
          siteUrl
        }
      }
    }
    """
    variables = {"page": page, "perPage": per_page}
    data = run_query(query, variables)
    return data["Page"]

def save_to_csv(anime_list, filename="anime_winter_2026.csv"):
    """将结果保存为 CSV"""
    with open(filename, "w", newline='', encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow([
            "ID", "Title", "Format", "Episodes", "Duration(min)", "Start Date",
            "Studio", "Genres", "Average Score", "Popularity",
            "Description", "AniList URL"
        ])
        for anime in anime_list:
            title = anime["title"]["userPreferred"]
            start = anime["startDate"]
            start_date = f"{start.get('year', '')}-{start.get('month', '')}-{start.get('day', '')}"
            studio = ", ".join([s["name"] for s in anime["studios"]["nodes"]]) if anime["studios"]["nodes"] else ""
            genres = ", ".join(anime.get("genres", []))
            desc = anime.get("description") or ""
            desc = desc.replace("\n", " ").replace("\r", " ")[:500]  # 保护 + 截断
            
            writer.writerow([
                anime["id"],
                title,
                anime.get("format", ""),
                anime.get("episodes", ""),
                anime.get("duration", ""),
                start_date,
                studio,
                genres,
                anime.get("averageScore", ""),
                anime.get("popularity", ""),
                desc,
                anime.get("siteUrl", "")
            ])

    print(f"💾 已保存至 {filename}")

if __name__ == "__main__":
    page = 1
    per_page = 50
    all_anime = []

    print("🔍 正在采集 2026 年 1 月（冬季）新番数据...")

    while True:
        result = get_winter_2026_anime(page, per_page)
        all_anime.extend(result["media"])
        if not result["pageInfo"]["hasNextPage"]:
            break
        page += 1

    print(f"✅ 共采集到 {len(all_anime)} 部新番")
    save_to_csv(all_anime)

🔍 正在采集 2026 年 1 月（冬季）新番数据...
✅ 共采集到 83 部新番
💾 已保存至 anime_winter_2026.csv


In [ ]:
import requests
import csv

API_URL = "https://graphql.anilist.co"

def run_query(query, variables=None):
    """执行 AniList GraphQL 查询"""
    headers = {"Content-Type": "application/json"}
    response = requests.post(API_URL, json={"query": query, "variables": variables or {}}, headers=headers)
    response.raise_for_status()
    data = response.json()
    if "errors" in data:
        raise Exception(data["errors"])
    return data["data"]

def get_winter_2026_anime(page=1, per_page=50):
    """获取 2026 年冬季（1月）新番"""
    query = """
    query ($page: Int, $perPage: Int) {
      Page(page: $page, perPage: $perPage) {
        pageInfo {
          hasNextPage
          currentPage
        }
        media(
          type: ANIME
          season: WINTER
          seasonYear: 2026
          sort: START_DATE
        ) {
          id
          title {
            romaji
            english
            native
            userPreferred
          }
          description(asHtml: false)
          format
          episodes
          duration
          startDate { year month day }
          studios(isMain: true) {
            nodes { name }
          }
          genres
          averageScore
          popularity
          siteUrl
        }
      }
    }
    """
    variables = {"page": page, "perPage": per_page}
    data = run_query(query, variables)
    return data["Page"]

def save_to_csv(anime_list, filename="anime_winter_2026.csv"):
    """将结果保存为 CSV"""
    with open(filename, "w", newline='', encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow([
            "ID", "Title", "Title_Japanese", "Format", "Episodes",
            "Duration(min)", "Start Date", "Studio", "Genres",
            "Average Score", "Popularity", "Description", "AniList URL"
        ])

        for anime in anime_list:
            title = anime["title"]["userPreferred"]

            # ★ 获取日文标题 native ★
            title_jp = anime["title"].get("native") or ""

            start = anime["startDate"]
            start_date = f"{start.get('year', '')}-{start.get('month', '')}-{start.get('day', '')}"

            studio = ", ".join([s["name"] for s in anime["studios"]["nodes"]]) \
                     if anime["studios"]["nodes"] else ""

            genres = ", ".join(anime.get("genres", []))
            desc = anime.get("description") or ""
            desc = desc.replace("\n", " ").replace("\r", " ")[:500]

            writer.writerow([
                anime["id"],
                title,
                title_jp,      # ★ 新增输出日文标题
                anime.get("format", ""),
                anime.get("episodes", ""),
                anime.get("duration", ""),
                start_date,
                studio,
                genres,
                anime.get("averageScore", ""),
                anime.get("popularity", ""),
                desc,
                anime.get("siteUrl", "")
            ])

    print(f"💾 已保存至 {filename}")

if __name__ == "__main__":
    page = 1
    per_page = 50
    all_anime = []

    print("🔍 正在采集 2026 年 1 月（冬季）新番数据...")

    while True:
        result = get_winter_2026_anime(page, per_page)
        all_anime.extend(result["media"])
        if not result["pageInfo"]["hasNextPage"]:
            break
        page += 1

    print(f"✅ 共采集到 {len(all_anime)} 部新番")
    save_to_csv(all_anime)


🔍 正在采集 2026 年 1 月（冬季）新番数据...
✅ 共采集到 83 部新番
💾 已保存至 anime_winter_2026.csv


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "e:\anaconda\envs\nlp_env\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "e:\anaconda\envs\nlp_env\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "e:\anaconda\envs\nlp_env\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "e:\anaconda\envs\nlp_env\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: 

In [18]:
df_oldanidata = pd.read_csv("anilist_anime_2016_2025.csv")
df_newanidata = pd.read_csv("anime_winter_2026.csv")

| 字段名                  | 说明                             | 示例                                                          |
| -------------------- | ------------------------------ | ----------------------------------------------------------- |
| `id`                 | AniList 动画条目 ID                | 11061                                                       |
| `idMal`              | MyAnimeList 对应 ID（可为空）         | 5114                                                        |
| `title_romaji`       | 罗马音标题                          | *Shingeki no Kyojin*                                        |
| `title_english`      | 英文标题                           | *Attack on Titan*                                           |
| `title_native`       | 日文标题                           | 進撃の巨人                                                       |
| `season`             | 放送季度                           | WINTER / SPRING / SUMMER / FALL                             |
| `seasonYear`         | 放送年份                           | 2019                                                        |
| `startDate`          | 开始日期                           | 2019-04-07                                                  |
| `endDate`            | 结束日期                           | 2019-09-29                                                  |
| `episodes`           | 总集数                            | 12                                                          |
| `duration`           | 每集时长（分钟）                       | 24                                                          |
| `format`             | 类型                             | TV / MOVIE / OVA / ONA 等                                    |
| `status`             | 当前状态                           | FINISHED / RELEASING / NOT_YET_RELEASED                     |
| `source`             | 改编来源                           | MANGA / NOVEL / ORIGINAL / GAME 等                           |
| `averageScore`       | 用户平均分（0–100）                   | 87                                                          |
| `meanScore`          | 平均得分                           | 85                                                          |
| `popularity`         | 人气值（总观众数或热度指标）                 | 230000                                                      |
| `favourites`         | 被收藏数量                          | 18000                                                       |
| `trending`           | 当前趋势热度（每日活跃度指标）                | 385                                                         |
| `genres`             | 类型标签（以“|”分隔）                   | Action|Drama|Fantasy                                        |
| `tags`               | 更细粒度标签（来自 AniList 内部分类）        | Gore|Shounen|Post-Apocalyptic                               |
| `mainStudio`         | 主制作工作室                         | MAPPA                                                       |
| `rankings_json`      | JSON 格式的排名信息（如季度/全时段排名）        | `[{"rank":1,"type":"RATED","year":2019,"season":"SPRING"}]` |
| `externalLinks_json` | JSON 格式的外部链接（如官方网站、PV、Twitter） | `[{"site":"YouTube","url":"https://youtu.be/..."}]`         |


In [20]:
df_newanidata['Start Date']

0     2025-12-None
1       2025-12-20
2       2025-12-26
3      2026-1-None
4      2026-1-None
          ...     
67        2026-3-6
68       2026-3-13
69       2026-3-20
70       2026-3-27
71     2026-4-None
Name: Start Date, Length: 72, dtype: object

In [22]:
df_oldanidata.head()

,id,idMal,title_romaji,title_english,title_native,season,seasonYear,startDate,endDate,episodes,...,averageScore,meanScore,popularity,favourites,trending,genres,tags,mainStudio,rankings_json,externalLinks_json
0,21234,31043.0,Boku dake ga Inai Machi,ERASED,僕だけがいない街,WINTER,2016,2016-1-8,2016-3-25,12.0,...,81.0,81.0,540259,19812,5,Drama|Mystery|Psychological|Supernatural|Thriller,Age Regression|Time Manipulation|Crime|Tragedy...,A-1 Pictures,"[{""rank"": 240, ""type"": ""RATED"", ""year"": null, ...","[{""site"": ""Official Site"", ""url"": ""http://boku..."
1,21202,30831.0,Kono Subarashii Sekai ni Shukufuku wo!,KONOSUBA -God's blessing on this wonderful world!,この素晴らしい世界に祝福を！,WINTER,2016,2016-1-14,2016-3-17,10.0,...,79.0,79.0,457132,18977,5,Adventure|Comedy|Ecchi|Fantasy,Isekai|Satire|Parody|Primarily Female Cast|Ens...,Studio DEEN,"[{""rank"": 365, ""type"": ""RATED"", ""year"": null, ...","[{""site"": ""Official Site"", ""url"": ""http://kono..."
2,21170,30654.0,Ansatsu Kyoushitsu 2nd Season,Assassination Classroom Second Season,暗殺教室 第２期,WINTER,2016,2016-1-8,2016-7-1,25.0,...,83.0,83.0,331050,9616,5,Action|Comedy|Drama|Supernatural,Assassins|School|Teacher|Coming of Age|Primari...,Lerche,"[{""rank"": 122, ""type"": ""RATED"", ""year"": null, ...","[{""site"": ""Official Site"", ""url"": ""http://www...."
3,21428,31859.0,Hai to Gensou no Grimgar,Grimgar of Fantasy and Ash,灰と幻想のグリムガル,WINTER,2016,2016-1-11,2016-3-28,12.0,...,74.0,74.0,161329,3493,2,Action|Adventure|Drama|Fantasy,Isekai|Survival|Tragedy|Goblin|Ensemble Cast|M...,A-1 Pictures,"[{""rank"": 288, ""type"": ""POPULAR"", ""year"": null...","[{""site"": ""Official Site"", ""url"": ""http://grim..."
4,9260,9260.0,Kizumonogatari I: Tekketsu-hen,Kizumonogatari Part 1: Tekketsu,傷物語〈Ⅰ鉄血篇〉,WINTER,2016,2016-1-8,2016-1-8,1.0,...,83.0,83.0,151546,3793,1,Action|Drama|Ecchi|Mystery|Psychological|Super...,Vampire|Urban Fantasy|Rotoscoping|Urban|Suicid...,Shaft,"[{""rank"": 47, ""type"": ""RATED"", ""year"": null, ""...","[{""site"": ""Official Site"", ""url"": ""http://www...."
